<a href="https://colab.research.google.com/github/thamyrissacamoto/MarvelAPI/blob/Thamyris%2FconsumindoAPI/client.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import hashlib
import time
import requests
from google.colab import userdata
import pandas as pd
from tqdm import tqdm


In [ ]:
BASE_URL = "https://gateway.marvel.com/v1/public/"
PUBLIC_KEY = userdata.get('public_key')
PRIVATE_KEY = userdata.get('private_key')

def get_auth():
    ts = str(time.time())
    to_hash = ts + PRIVATE_KEY + PUBLIC_KEY
    hash_result = hashlib.md5(to_hash.encode()).hexdigest()
    return {
        "ts": ts,
        "apikey": PUBLIC_KEY,
        "hash": hash_result
    }


In [ ]:
etag_cache = {}

def marvel_request(endpoint, params_extra={}):
    url = BASE_URL + endpoint
    auth_params = get_auth()
    params = auth_params.copy()
    params.update(params_extra)

    cache_key = f"{endpoint}_{params_extra}"

    headers = {}
    if cache_key in etag_cache:
        headers["If-None-Match"] = etag_cache[cache_key]["etag"]

    response = requests.get(url, params=params, headers=headers)

    if response.status_code == 304:
      print(response.status_code)
      return etag_cache[cache_key]["data"]
    elif response.status_code == 200:
      print(response.status_code)
      etag = response.headers.get("etag")
      data = response.json()
      etag_cache[cache_key] = {"etag": etag, "data": data}
      return data
    else:
      print(f"{response.status_code}: {response.text}")


In [ ]:
data = marvel_request("characters", {"limit": 5, "offset": 0})

df = pd.DataFrame(data)
df_normalizado = pd.json_normalize(df["data"]["results"])
df_normalizado.head()

In [ ]:
def get_all_data(endpoint, columns={}):
    """
    Recupera todos os dados de um endpoint da Marvel API usando paginação.
    Retorna um DataFrame com os resultados combinados.
    """

    first_response = marvel_request(endpoint, {"limit": 100, "offset": 0})
    if not first_response:
        print("Erro ao obter dados iniciais.")
        return None

    total = first_response["data"]["total"]
    all_results = first_response["data"]["results"]

    for offset in tqdm(range(100, total, 100), desc=f"Baixando dados de {endpoint}"):
        response = marvel_request(endpoint, {"limit": 100, "offset": offset})
        if response:
            all_results.extend(response["data"]["results"])

    df = pd.json_normalize(all_results)
    return df

In [ ]:
endpoints = ["characters", "comics", "creators", "events", "series", "stories"]

for endpoint in endpoints:
    print(f"\nProcessando dados de: {endpoint}")
    df = get_all_data(endpoint)
    df.to_csv(f"{endpoint}_raw.csv", index=False)
